In [25]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import pathlib
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils
import os
print(tf.__version__)


2.0.0


In [26]:
train_dataset = np.loadtxt('train.csv',skiprows = 1,delimiter = ',')
test_dataset = np.loadtxt('test.csv',skiprows = 1,delimiter = ',')

In [27]:
train_dataset[:5]

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
 x_train = train_dataset[:, 1:]

In [29]:
x_train[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
x_train.shape

(42000, 784)

In [49]:
x_train = x_train.reshape(x_train.shape[0],28,28,1)

In [50]:
x_train /= 255.0

In [51]:
y_train = train_dataset[:, 0]

In [52]:
x_train.shape

(42000, 28, 28, 1)

In [53]:
data_generator = ImageDataGenerator(
rotation_range = 10,
zoom_range = 0.10,
width_shift_range = 0.1,
height_shift_range = 0.1)

In [54]:
y_train = utils.to_categorical(y_train)

In [55]:
random_seed = 2
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train, test_size = 0.1, random_state=random_seed)

In [56]:
y_train[:5]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [57]:
batch_size=96

In [40]:
x = data_generator.flow(X_train,Y_train, batch_size=batch_size)

In [41]:
type(x)

keras_preprocessing.image.numpy_array_iterator.NumpyArrayIterator

In [42]:
print(x)

In [61]:
model = Sequential()

model.add(layers.Conv2D(32, (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(layers.Conv2D(32, (5,5),padding = 'Same', activation ='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))


model.add(layers.Conv2D(64, (3,3),padding = 'Same', activation ='relu'))
model.add(layers.Conv2D(64, (3,3),padding = 'Same', activation ='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = "relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation = "softmax"))

In [62]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 64)         

In [63]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
сheckpoint = ModelCheckpoint(filepath=checkpoint_path,
                             monitor='val_acc',
                              save_best_only=True,
                              verbose=1)

In [64]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [29]:
history = model.fit(data_generator.flow(X_train,Y_train, batch_size=batch_size), 
                    epochs=10,
                    validation_data=(X_valid, Y_valid),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    verbose=1,
                    callbacks=[сheckpoint, learning_rate_reduction])

Train for 393 steps, validate on 4200 samples
Epoch 1/10
393/393 [==============================] - 616s 2s/step - loss: 0.4277 - accuracy: 0.8652 - val_loss: 0.0598 - val_accuracy: 0.9807
Epoch 2/10
393/393 [==============================] - 642s 2s/step - loss: 0.1386 - accuracy: 0.9598 - val_loss: 0.0431 - val_accuracy: 0.9862
Epoch 3/10
393/393 [==============================] - 626s 2s/step - loss: 0.0994 - accuracy: 0.9704 - val_loss: 0.0364 - val_accuracy: 0.9883
Epoch 4/10
393/393 [==============================] - 618s 2s/step - loss: 0.0865 - accuracy: 0.9748 - val_loss: 0.0314 - val_accuracy: 0.9912
Epoch 5/10
393/393 [==============================] - 617s 2s/step - loss: 0.0759 - accuracy: 0.9769 - val_loss: 0.0318 - val_accuracy: 0.9910
Epoch 6/10
393/393 [==============================] - 613s 2s/step - loss: 0.0685 - accuracy: 0.9796 - val_loss: 0.0329 - val_accuracy: 0.9914
Epoch 7/10
393/393 [==============================] - 610s 2s/step - loss: 0.0628 - accuracy: 0.

In [68]:
model = keras.models.load_model('recogn_numbers.h5')

In [30]:
model.save('recogn_numbers.h5')

In [32]:
test_dataset = np.loadtxt('test.csv', skiprows=1, delimiter=",")


In [33]:
x_test = test_dataset.reshape(test_dataset.shape[0], 28, 28, 1)

In [34]:
x_test = x_test / 255.0

In [35]:
predictions = model.predict(x_test)

In [36]:
predictions = np.argmax(predictions, axis=1)

In [38]:
out = np.column_stack((range(1, predictions.shape[0]+1), predictions))

In [39]:
out[:5]

array([[1, 2],
       [2, 0],
       [3, 9],
       [4, 0],
       [5, 3]], dtype=int64)

In [40]:
np.savetxt('submission.csv', out, header="ImageId,Label", 
            comments="", fmt="%d,%d")